In [6]:
#importing the libraries
import numpy as np
import sys
import os
import codecs

In [7]:
#initialising the tags
tags = ['NN', 'NST', 'NNP', 'PRP', 'DEM', 'VM', 'VAUX', 'JJ', 'RB', 'PSP', 'RP', 'CC', 'WQ', 'QF', 'QC', 'QO', 'CL', 'INTF', 'INJ', 'NEG', 'UT', 'SYM', 'COMP', 'RDP', 'ECH', 'UNK', 'XC']

In [8]:
#This function performs the viterbi decoding. Choosing which tag for the current word leads to a better tag sequence
def max_connect(x, y, viterbi_matrix, emission, transmission_matrix):
	max = -99999
	path = -1
	
	for i in range(len(tags)):
		val = viterbi_matrix[i][x-1] * transmission_matrix[i][y]
		if val * emission > max:
			max = val
			path = i
	return max, path

In [9]:
#initialising certain variables
extras = ["<s>", "</s>", "START", "END"]
word_type = []
pos_tag_count = []

In [28]:
#loading and reading the dataset
f = open("/content/tamil_training.txt", "r")
filecontent=f.readlines()
print(len(filecontent))
print(filecontent)

9151
['<s> START\n', 'இது PRP\n', 'தவறு NN\n', 'என UT\n', 'நீங்கள் PRP\n', 'கருதினால் VM\n', 'இங்கே RB\n', 'தெரிவியுங்கள் VM\n', 'அல்லது CC\n', 'எனக்கு PRP\n', 'மின்னஞ்சல் NN\n', 'செய்யுங்கள் VM\n', '</s> END\n', '<s> START\n', 'தொழில்நுட்பம் NN\n', 'தொடர்புடைய VM\n', 'கட்டுரைகள் NN\n', 'இந்த DEM\n', 'பக்க JJ\n', 'வகையில் NN\n', 'அடங்கும் VM\n', '</s> END\n', '<s> START\n', 'மேலே RB\n', 'குறிப்பிட்ட JJ\n', 'எண்களை NN\n', 'ஒன்று NN\n', 'இரண்டு QC\n', 'மூன்று QC\n', 'நான்கு QC\n', 'ஐந்து QC\n', '</s> END\n', '<s> START\n', 'சிந்து NN\n', 'நதி NN\n', 'இந்தியா NNP\n', 'பாகிஸ்தான் NNP\n', 'ஆகிய JJ\n', 'நாடுகளில் NN\n', 'ஓடும் VM\n', 'முக்கிய JJ\n', 'நதி NN\n', '</s> END\n', '<s> START\n', 'கண்டி NN\n', 'நகரில் NN\n', 'உள்ள PSP\n', 'புகழ் NN\n', 'பெற்ற VM\n', 'பௌத்த JJ\n', 'ஆலயம் NN\n', 'ஆகும் VM\n', '</s> END\n', '<s> START\n', 'ஸ்ரீநிவாசன் NNP\n', 'இயக்கத்தில் NN\n', 'வெளிவந்த VM\n', 'இத்திரைப்படத்தில் NN\n', 'சிவாஜி NN\n', 'கணேசன் NNP\n', 'கே NN\n', '. SYM\n', 'ஆர் NN\n', '. SYM\n', 'விஜய

In [11]:
#initialising the count of each tag to 0
for i in range(len(tags)):
		pos_tag_count.append(0)

In [12]:
#calculate the count of each tag and the word type in the training corpus
for i in range(len(filecontent)):
		line = filecontent.pop(0).strip().split(' ')
		#print(line)
		for i, word in enumerate(line):
			if i == 0:
				if word not in word_type and word not in extras:
					word_type.append(word)
			else:
				if word in tags and word not in extras:
					pos_tag_count[tags.index(word)] += 1

In [13]:
line

['</s>', 'END']

In [14]:
#Defining variables for emission and transmission matrix
emission_prob_matrix = []
transmission_prob_matrix = []

In [15]:
#initialise emission matrix
for i in range(len(tags)):
		emission_prob_matrix.append([])
		for j in range(len(word_type)):
			emission_prob_matrix[i].append(0)

In [16]:
#initialise transition matrix
for i in range(len(tags)):
		transmission_prob_matrix.append([])
		for j in range(len(tags)):
			transmission_prob_matrix[i].append(0)

In [17]:
#update the training and transition matrix to the appropriate counts
row_id = -1
for i in range(len(filecontent)):
		line = filecontent.pop(0).strip().split(' ')
		
		if line[0] not in extras:
			col_id = word_type.index(line[0])
			prev_row_id = row_id
			row_id = tags.index(line[1])
			emission_prob_matrix[row_id][col_id] += 1
			if prev_row_id != -1:
				transmission_prob_matrix[prev_row_id][row_id] += 1
		else:
			row_id = -1

In [18]:
word_type

['இது',
 'தவறு',
 'என',
 'நீங்கள்',
 'கருதினால்',
 'இங்கே',
 'தெரிவியுங்கள்',
 'அல்லது',
 'எனக்கு',
 'மின்னஞ்சல்',
 'செய்யுங்கள்',
 'தொழில்நுட்பம்',
 'தொடர்புடைய',
 'கட்டுரைகள்',
 'இந்த',
 'பக்க',
 'வகையில்',
 'அடங்கும்',
 'மேலே',
 'குறிப்பிட்ட',
 'எண்களை',
 'ஒன்று',
 'இரண்டு',
 'மூன்று',
 'நான்கு',
 'ஐந்து',
 'சிந்து',
 'நதி',
 'இந்தியா',
 'பாகிஸ்தான்',
 'ஆகிய',
 'நாடுகளில்',
 'ஓடும்',
 'முக்கிய',
 'கண்டி',
 'நகரில்',
 'உள்ள',
 'புகழ்',
 'பெற்ற',
 'பௌத்த',
 'ஆலயம்',
 'ஆகும்',
 'ஸ்ரீநிவாசன்',
 'இயக்கத்தில்',
 'வெளிவந்த',
 'இத்திரைப்படத்தில்',
 'சிவாஜி',
 'கணேசன்',
 'கே',
 '.',
 'ஆர்',
 'விஜயா',
 'மற்றும்',
 'பலரும்',
 'நடித்துள்ளனர்',
 'எழுத்துப்',
 'பிழை',
 'திருத்தம்',
 'செய்யும்',
 'போது',
 'பாலச்சந்தர்',
 'ரஜினிகாந்த்',
 'கமல்ஹாசன்',
 'கிருஷ்ணன்',
 'பஞ்சு',
 'ராஜேந்திரன்',
 'யாருக்கும்',
 'மாற்றுக்',
 'கருத்து',
 'இருந்தால்',
 'பின்னர்',
 'மாற்றிக்',
 'கொள்ளலாம்',
 'நிலையில்',
 'முன்னேற்றம்',
 'ஏற்பட்டு',
 'வருகின்றது',
 'நினைக்கிறேன்',
 'இராம',
 'கி',
 'செல்வா',
 'போன்றவர்கள்',
 '

In [19]:
#update the emission probability. This piece of code was taken from https://towardsdatascience.com/implementing-part-of-speech-tagging-for-english-words-using-viterbi-algorithm-from-scratch-9ded56b29133
for i in range(len(tags)):
		for j in range(len(word_type)):
			if pos_tag_count[i] != 0:
				emission_prob_matrix[i][j] = float(emission_prob_matrix[i][j]) / pos_tag_count[i]
				#print(emission_prob_matrix[i][j])

In [20]:
#update the transition probability. This piece of code was taken from https://towardsdatascience.com/implementing-part-of-speech-tagging-for-english-words-using-viterbi-algorithm-from-scratch-9ded56b29133
for i in range(len(tags)):
		for j in range(len(tags)):
			if pos_tag_count[i] != 0:
				transmission_prob_matrix[i][j] = float(transmission_prob_matrix[i][j]) / pos_tag_count[i]

In [29]:
#open the test file
f1= open("/content/tamil_testing.txt", "r")
filecontentest=f1.readlines()
print(len(filecontentest))
print(filecontentest)

100
['அது குறித்த தகவல்களை தேடி எடுத்து தருகிறேன்\n', 'தலித் இலக்கியம் பற்றி பல கட்டுரை எழுதியுள்ளார்\n', 'இம்மொழி வங்காள தேசம் மற்றும் இந்தியாவின்\n', 'ஏன் என்றால் உங்கள் ஆங்கிலம் நன்று\n', 'எனவே லைடன் நெதர்லாந்து பொருத்தம் என்றே தோன்றுகின்றது\n', 'கருநாடக இசை அறிஞர்கள் குறித்த கட்டுரையைத் தொடங்குங்கள்\n', 'முத்துராமன் இயக்கத்தில் வெளிவந்த இத்திரைப்படத்தில் முத்துராமன் ஜெயலலிதா மற்றும் பலரும் நடித்துள்ளனர்\n', 'இக்கட்டுரை குறித்து உங்களிடம் மேலும் தகவல்கள் இருந்தால் அருள்கூர்ந்து எனக்கு தெரிவிக்கவும்\n', 'இது ஒரு பெரிய பிரச்சினை இல்லை என்று நினைக்கின்றேன்\n', 'வைணவ இலக்கியங்கள் பற்றி பிற கட்டுரைகளில் தேடிப்பாருங்கள்\n', 'எனது ஆங்கில விக்கிபீடியா பயனர் பக்கம்\n', 'தற்போதைய நிலைமை ஒரு தொடக்க புள்ளி மட்டுமே\n', 'இதன் எல்லைகளாக யாழ்ப்பாணம் கிளிநொச்சி மன்னார் வவுனியா திருகோணமலை விளங்குகின்றன\n', 'இதை நான் உங்களுக்கு சொல்ல வேண்டியதில்லை\n', 'தங்கள் வரவு தமிழ் விக்கிபீடியாவிற்கு மேலும் பலம் சேர்க்கும் என நம்புகிறேன்\n', 'தற்போது தனியார் வானொலி நிலையம் ஒன்றில் பணியாற்றி வருகிறார்\n', 'இந்த தல

In [22]:
test_words = []
pos_tags = []

In [23]:
#model training. This piece of code was taken from https://towardsdatascience.com/implementing-part-of-speech-tagging-for-english-words-using-viterbi-algorithm-from-scratch-9ded56b29133
for j in range(len(filecontentest)):
  test_words=[]
  pos_tags=[]
  line1=filecontentest.pop(0).strip().split(' ')
  #print(line1)
  for word in line1:
    test_words.append(word)
    pos_tags.append(-1)
    #print(test_words)

    viterbi_matrix=[]
    viterbi_path=[]

  for x in range(len(tags)):
    viterbi_matrix.append([])
    viterbi_path.append([])
    for y in range(len(test_words)):
      viterbi_matrix[x].append(0)
      viterbi_path[x].append(0)
      #print(viterbi_matrix[x])

  for x in range(len(test_words)):
    for y in range(len(tags)):
      if test_words[x] in word_type:
        word_index=word_type.index(test_words[x])
        tag_index=tags.index(tags[y])
        emission=emission_prob_matrix[tag_index][word_index]
        #print(emission)
      else:
        emission=0.001

      if x > 0:
        max, viterbi_path[y][x] = max_connect(x, y, viterbi_matrix, emission, transmission_prob_matrix)
        #print(max)
      else:
        max = 1
        viterbi_matrix[y][x] = emission * max

  maxval = -999999
  maxs = -1
  for x in range(len(tags)):
    if viterbi_matrix[x][len(test_words)-1] > maxval:
      maxval = viterbi_matrix[x][len(test_words)-1]
      maxs = x
    for x in range(len(test_words)-1, -1, -1):
      pos_tags[x] = maxs
      #print(pos_tags)
      maxs = viterbi_path[maxs][x]
      #print(maxs)
  with open('resultfinal2.txt', 'w') as f:
    for i, x in enumerate(pos_tags):
      print(test_words[i] + "_" + tags[x] + " ")
  
  test_run_base = [tup for tup in test_words]
  #print(test_run_base)

அது_NN 
குறித்த_NN 
தகவல்களை_NN 
தேடி_NN 
எடுத்து_NN 
தருகிறேன்_NN 
தலித்_NN 
இலக்கியம்_NN 
பற்றி_NN 
பல_NN 
கட்டுரை_NN 
எழுதியுள்ளார்_NN 
இம்மொழி_NN 
வங்காள_NN 
தேசம்_NN 
மற்றும்_NN 
இந்தியாவின்_NN 
ஏன்_NN 
என்றால்_NN 
உங்கள்_NN 
ஆங்கிலம்_NN 
நன்று_NN 
எனவே_NN 
லைடன்_NN 
நெதர்லாந்து_NN 
பொருத்தம்_NN 
என்றே_NN 
தோன்றுகின்றது_NN 
கருநாடக_NN 
இசை_NN 
அறிஞர்கள்_NN 
குறித்த_NN 
கட்டுரையைத்_NN 
தொடங்குங்கள்_NN 
முத்துராமன்_NN 
இயக்கத்தில்_NN 
வெளிவந்த_NN 
இத்திரைப்படத்தில்_NN 
முத்துராமன்_NN 
ஜெயலலிதா_NN 
மற்றும்_NN 
பலரும்_NN 
நடித்துள்ளனர்_NN 
இக்கட்டுரை_NN 
குறித்து_NN 
உங்களிடம்_NN 
மேலும்_NN 
தகவல்கள்_NN 
இருந்தால்_NN 
அருள்கூர்ந்து_NN 
எனக்கு_NN 
தெரிவிக்கவும்_NN 
இது_NN 
ஒரு_NN 
பெரிய_NN 
பிரச்சினை_NN 
இல்லை_NN 
என்று_NN 
நினைக்கின்றேன்_NN 
வைணவ_NN 
இலக்கியங்கள்_NN 
பற்றி_NN 
பிற_NN 
கட்டுரைகளில்_NN 
தேடிப்பாருங்கள்_NN 
எனது_NN 
ஆங்கில_NN 
விக்கிபீடியா_NN 
பயனர்_NN 
பக்கம்_NN 
தற்போதைய_NN 
நிலைமை_NN 
ஒரு_NN 
தொடக்க_NN 
புள்ளி_NN 
மட்டுமே_NN 
இதன்_NN 
எல்லைகளாக_NN 
யாழ்ப்பாணம்_NN 
கிள

In [24]:
accuracy = len(pos_tags)/len(test_run_base)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Viterbi Algorithm Accuracy:  100.0
